In [ ]:
import torch 
import torch.nn as nn
import torch.optim as optim 
import torch.nn.functional as F
import random 
from torch.utils.data import DataLoader, Dataset
from sklearn.datasets import load_iris, load_digits
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import numpy as np

In [ ]:
class SensorNeuron(nn.Module): 
    def __init__(self, input_size, dataset_samples):
        super(SensorNeuron, self).__init__() 
        self.input_size = input_size 
        self.dataset_samples = dataset_samples
        self.sensory_db = self.initialize_sensory_neurons()
        self.consolidation_counter = [0] * input_size 
        # Consolidation counters
    def initialize_sensory_neurons(self): 
        # Initialize with weights randomly derived from dataset entries
        sensory_neurons = [] 
        for _ in range(self.input_size):
            random_sample = random.choice(self.dataset_samples)
            sensory_neurons.append(random.choice(random_sample)) 
        return torch.tensor(sensory_neurons).float() 
        
    def forward(self, x):
        x = x.float()
        reinforcement_score = 0
        for i in range(len(self.sensory_db)):
            # Check if the feature passes through the neuron using torch.isclose 
            if torch.all(torch.isclose(x[:, i], torch.tensor([self.sensory_db[i]] * x.size(0)), atol=1e-5)):
                self.consolidation_counter[i] += 1 
                # Increment consolidation counter 
                reinforcement_score += 1 
            else:
                self.consolidation_counter[i] -= 1 
                # Decrement counter if doesn't match 
                # Replace the characteristic if the counter becomes negative 
            if self.consolidation_counter[i] < 0:
                random_sample = random.choice(self.dataset_samples)
                self.sensory_db[i] = random.choice(random_sample) 
                self.consolidation_counter[i] = 0
                
        return x, reinforcement_score

In [ ]:
class AdvancedMLP(nn.Module):
    def __init__(self, input_size, hidden_size, output_size, dataset_samples, dropout_rate=0.5):
        super(AdvancedMLP, self).__init__()
        self.sensor_neuron = SensorNeuron(input_size=input_size, dataset_samples=dataset_samples) 
        self.fc1 = nn.Linear(input_size, hidden_size) 
        self.bn1 = nn.BatchNorm1d(hidden_size) 
        self.dropout = nn.Dropout(dropout_rate)
        self.fc2 = nn.Linear(hidden_size, output_size) 
        
    def forward(self, x):
        x, reinforcement_score = self.sensor_neuron(x) 
        x = F.relu(self.bn1(self.fc1(x)))
        x = self.dropout(x) 
        x = self.fc2(x)
        return F.log_softmax(x, dim=1)

In [ ]:
# Carregar o dataset Iris
iris = load_iris() 
x_iris = iris.data
y_iris = iris.target
# Dividir os dados em conjuntos de treinamento e teste
x_treino_iris, x_teste_iris, y_treino_iris, y_teste_iris = train_test_split(x_iris, y_iris, test_size=0.3, random_state=42) 
# Normalizar os dados
scaler = StandardScaler() 
x_treino_iris = scaler.fit_transform(x_treino_iris)
x_teste_iris = scaler.transform(x_teste_iris) 
# Converter para tensores do PyTorch 
x_treino_iris = torch.tensor(x_treino_iris, dtype=torch.float32)
y_treino_iris = torch.tensor(y_treino_iris, dtype=torch.long) 
x_teste_iris = torch.tensor(x_teste_iris, dtype=torch.float32)
y_teste_iris = torch.tensor(y_teste_iris, dtype=torch.long) 
# Parâmetros do modelo 
input_size = x_iris.shape[1] 
hidden_size = 50 
output_size = len(np.unique(y_iris))

In [ ]:
# Carregar dataset Iris 
iris = load_iris() 
X = iris.data 
y = iris.target 
# Dividir o dataset em treino e teste
x_treino_iris, x_teste_iris, y_treino_iris, y_teste_iris = train_test_split(X, y, test_size=0.2, random_state=42) 
# Converter para tensores 
x_treino_iris = torch.tensor(x_treino_iris, dtype=torch.float32)
x_teste_iris = torch.tensor(x_teste_iris, dtype=torch.float32)
y_treino_iris = torch.tensor(y_treino_iris, dtype=torch.long)
y_teste_iris = torch.tensor(y_teste_iris, dtype=torch.long) 
# Definir hiperparâmetros 
input_size = X.shape[1] 
hidden_size = 10 
output_size = len(set(y)) 
# Criar e treinar o modelo
model = AdvancedMLP(input_size, hidden_size, output_size, dataset_samples=x_treino_iris.tolist()) 
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001) 
# Treinamento 
num_epochs = 1000 
for epoch in range(num_epochs): 
    model.train() 
    optimizer.zero_grad()
    outputs = model(x_treino_iris) 
    loss = criterion(outputs, y_treino_iris) 
    loss.backward() 
    optimizer.step()
    if (epoch + 1) % 10 == 0:
        print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}') 


In [ ]:
# Avaliar o modelo 
model.eval()
with torch.no_grad():
    outputs = model(x_teste_iris)
    _, predicted = torch.max(outputs, 1)
    accuracy = (predicted == y_teste_iris).sum().item() / y_teste_iris.size(0) 
    print(f'Acurácia no conjunto de teste: {accuracy:.4f}')

In [ ]:
# Carregar dataset Iris 
digits = load_digits() 
X = digits.data 
y = digits.target 
# Dividir o dataset em treino e teste
x_treino_digits, x_teste_digits, y_treino_digits, y_teste_digits = train_test_split(X, y, test_size=0.2, random_state=42) 
# Converter para tensores 
x_treino_digits = torch.tensor(x_treino_digits, dtype=torch.float32)
x_teste_digits = torch.tensor(x_teste_digits, dtype=torch.float32)
y_treino_digits = torch.tensor(y_treino_digits, dtype=torch.long)
y_teste_digits = torch.tensor(y_teste_digits, dtype=torch.long) 
# Definir hiperparâmetros 
input_size = X.shape[1] 
hidden_size = 10 
output_size = len(set(y)) 
# Criar e treinar o modelo
model = AdvancedMLP(input_size, hidden_size, output_size, dataset_samples=x_treino_digits.tolist()) 
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001) 
# Treinamento 
num_epochs = 1000 
for epoch in range(num_epochs): 
    model.train() 
    optimizer.zero_grad()
    outputs = model(x_treino_digits) 
    loss = criterion(outputs, y_treino_digits) 
    loss.backward() 
    optimizer.step()
    if (epoch + 1) % 10 == 0:
        print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}') 

In [ ]:
# Avaliar o modelo 
model.eval()
with torch.no_grad():
    outputs = model(x_teste_digits)
    _, predicted = torch.max(outputs, 1)
    accuracy = (predicted == y_teste_digits).sum().item() / y_teste_digits.size(0) 
    print(f'Acurácia no conjunto de teste: {accuracy:.4f}')

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F 
import torch.optim as optim
from sklearn.datasets import load_digits 
from sklearn.model_selection import train_test_split
from torch.utils.data import DataLoader, Dataset 
import random 
# Carregar o conjunto de dados de dígitos 
digits = load_digits()
X = digits.images
y = digits.target 
# Redimensionar as imagens para [n_samples, 1, 8, 8] X = X[:, np.newaxis, :, :] 
# Dividir o conjunto de dados em treino e teste 
x_treino, x_teste, y_treino, y_teste = train_test_split(X, y, test_size=0.2, random_state=42) 
# Converter para tensores 
x_treino = torch.tensor(x_treino, dtype=torch.float32)
x_teste = torch.tensor(x_teste, dtype=torch.float32)
y_treino = torch.tensor(y_treino, dtype=torch.long) 
y_teste = torch.tensor(y_teste, dtype=torch.long) 
# Criar o Dataset e DataLoader
class DigitsDataset(Dataset):
    def __init__(self, images, labels):
        self.images = images 
        self.labels = labels 
        
    def __len__(self):
        return len(self.images)
        
    def __getitem__(self, idx):
        return self.images[idx], self.labels[idx]

    
train_dataset = DigitsDataset(x_treino, y_treino)
test_dataset = DigitsDataset(x_teste, y_teste) 
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

In [ ]:
# Definir a arquitetura da CNN 
class CNN(nn.Module):
    def __init__(self, dataset_samples):
        super(CNN, self).__init__() 
        self.sensor_neuron = SensorNeuron(input_size=64, dataset_samples=dataset_samples) 
        self.conv1 = nn.Conv2d(1, 32, kernel_size=3, padding=1) 
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, padding=1)
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2) 
        self.fc1 = nn.Linear(64 * 4 * 4, 128)
        self.fc2 = nn.Linear(128, 10)
        
    def forward(self, x):
        x, reinforcement_score = self.sensor_neuron(x) 
        x = self.pool(F.relu(self.conv1(x))) 
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(-1, 64 * 4 * 4) 
        x = F.relu(self.fc1(x)) 
        x = self.fc2(x) 
        return F.log_softmax(x, dim=1)

In [ ]:
# Criar o modelo, a função de perda e o otimizador 
model = CNN(dataset_samples=x_treino.tolist()) 
criterion = nn.CrossEntropyLoss() 
optimizer = optim.Adam(model.parameters(), lr=0.001)
# Treinar o modelo 
num_epochs = 10 
for epoch in range(num_epochs):
    model.train() 
    total_loss = 0 
    for images, labels in train_loader:
        optimizer.zero_grad() 
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward() 
        optimizer.step()
        total_loss += loss.item() 
    avg_loss = total_loss / len(train_loader) 
    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {avg_loss:.4f}')